## Twitter Sentiment Analysis with Gensim Word2Vec and Keras Convolutional Network
Giuseppe Bonaccorso (https://www.bonaccorso.eu)

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [1]:
import dill
import keras.backend as K
import multiprocessing
import tensorflow as tf

from gensim.models.word2vec import Word2Vec

from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv1D
from keras.optimizers import Adam

from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize import RegexpTokenizer

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
W1029 20:59:02.402895 26968 __init__.py:308] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
# Set random seed (for reproducibility)
import numpy as np
np.random.seed(1000)

Download the dataset from: http://thinknook.com/wp-content/uploads/2012/09/Sentiment-Analysis-Dataset.zip

In [5]:
dataset_location = 'Sentiment Analysis Dataset.csv'
#model_location = './model/'

### Parse tweet corpus and sentiments

In [6]:
corpus = []
labels = []

In [7]:
with open(dataset_location, 'r', encoding='utf-8') as df:
    for i, line in enumerate(df):
        if i == 0:
            # Skip the header
            continue

        parts = line.strip().split(',')
        
        # Sentiment (0 = Negative, 1 = Positive)
        labels.append(int(parts[1].strip()))
        
        # Tweet
        tweet = parts[3].strip()
        if tweet.startswith('"'):
            tweet = tweet[1:]
        if tweet.endswith('"'):
            tweet = tweet[::-1]
        
        corpus.append(tweet.strip().lower())
        
print('Corpus size: {}'.format(len(corpus)))

Corpus size: 1578627


### Tokenize and remove stopwords

In [12]:
tkr = RegexpTokenizer('[a-zA-Z0-9@]+')
stemmer = LancasterStemmer()

In [13]:
tokenized_corpus = []

for i, tweet in enumerate(corpus):
    tokens = [stemmer.stem(t) for t in tkr.tokenize(tweet) if not t.startswith('@')]
    tokenized_corpus.append(tokens)

#### Save tokenized corpus

In [14]:
with open('tokenized_corpus.dill', 'wb') as f:
    dill.dump(tokenized_corpus, f)

#### Load tokenized corpus

In [15]:
with open('tokenized_corpus.dill', 'rb') as f:
    tokenized_corpus = dill.load(f)

### Gensim Word2Vec model

In [16]:
vector_size = 512
window_size = 10

In [17]:
# Create Word2Vec
word2vec = Word2Vec(sentences=tokenized_corpus,
                    size=vector_size, 
                    window=window_size, 
                    negative=20,
                    iter=50,
                    seed=1000,
                    workers=multiprocessing.cpu_count())

#### Save Word2Vec model

In [18]:
word2vec.save('word2vec.model')

#### Load Word2Vec model

In [19]:
word2vec = Word2Vec.load( 'word2vec.model')

### Copy word vectors and delete Word2Vec model  and original corpus to save memory

In [20]:
X_vecs = word2vec.wv

del word2vec
del corpus

#### Train subset size (0 < size < len(tokenized_corpus))

In [21]:
train_size = 1000000

#### Test subset size (0 < size < len(tokenized_corpus) - train_size)

In [22]:
test_size = 100000

#### Compute average and max tweet length

In [53]:
avg_length = 0.0
max_length = 0

for tweet in tokenized_corpus:
    if len(tweet) > max_length:
        max_length = len(tweet)
    avg_length += float(len(tweet))
    
print('Average tweet length: {}'.format(avg_length / float(len(tokenized_corpus))))
print('Max tweet length: {}'.format(max_length))

Average tweet length: 11.02887762593697
Max tweet length: 53


#### Tweet max length (number of tokens)

In [54]:
max_tweet_length = 15

### Create train and test sets

In [52]:
# Create train and test sets
# Generate random indexes
indexes = set(np.random.choice(len(tokenized_corpus), train_size + test_size, replace=False))

X_train = np.zeros((train_size, max_tweet_length, vector_size), dtype=K.floatx())
Y_train = np.zeros((train_size, 2), dtype=np.int32)
X_test = np.zeros((test_size, max_tweet_length, vector_size), dtype=K.floatx())
Y_test = np.zeros((test_size, 2), dtype=np.int32)

for i, index in enumerate(indexes):
    for t, token in enumerate(tokenized_corpus[index]):
        if t >= max_tweet_length:
            break
        
        if token not in X_vecs:
            continue
    
        if i < train_size:
            X_train[i, t, :] = X_vecs[token]
        else:
            X_test[i - train_size, t, :] = X_vecs[token]
            
    if i < train_size:
        Y_train[i, :] = [1.0, 0.0] if labels[index] == 0 else [0.0, 1.0]
    else:
        Y_test[i - train_size, :] = [1.0, 0.0] if labels[index] == 0 else [0.0, 1.0]

MemoryError: 

### Shallow Neural Network

### Keras Convolutional model

In [47]:
batch_size = 32
nb_epochs = 10

In [49]:
model = Sequential()

model.add(Conv1D(32, kernel_size=3, activation='elu', padding='same', input_shape=(max_tweet_length, vector_size)))
model.add(Conv1D(32, kernel_size=3, activation='elu', padding='same'))
model.add(Conv1D(32, kernel_size=3, activation='elu', padding='same'))
model.add(Conv1D(32, kernel_size=3, activation='elu', padding='same'))
model.add(Dropout(0.25))

model.add(Conv1D(32, kernel_size=2, activation='elu', padding='same'))
model.add(Conv1D(32, kernel_size=2, activation='elu', padding='same'))
model.add(Conv1D(32, kernel_size=2, activation='elu', padding='same'))
model.add(Conv1D(32, kernel_size=2, activation='elu', padding='same'))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256, activation='tanh'))
model.add(Dense(256, activation='tanh'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

TypeError: 'int' object is not iterable

In [46]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001, decay=1e-6),
              metrics=['accuracy'])

In [48]:
# Fit the model
model.fit(X_train, Y_train,
          batch_size=batch_size,
          shuffle=True,
          epochs=nb_epochs,
          validation_data=(X_test, Y_test),
          callbacks=[EarlyStopping(min_delta=0.00025, patience=2)])

ValueError: Error when checking input: expected conv1d_1_input to have 3 dimensions, but got array with shape (1000000, 15)